In [3]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import matplotlib.pyplot as plt


In [ ]:
user = pd.read_csv('./userFeature.csv')
adstatic = pd.read_csv('./adStaticFeature.csv',header=0) 

In [ ]:
test = pd.read_csv('./data/preprocessed/newtestSample.csv')

test['pricepersize'] = test['price']/test['size'] * 100000000000000000
test['bigprice'] = test['price'] * 100000000000000000
test.rename(columns={'auid':'uid'}, inplace= True)
test = pd.merge(test, user, on='uid', how='left').drop_duplicates(subset=['aid','price'])
test

In [ ]:
statuscut = test['status'].str.split(" ", expand = True)
workcut = test['work'].str.split(" ", expand = True) 
test['status']=statuscut[0]
test['status1']=statuscut[1]
test['work']=workcut[0]
test['work1']=workcut[1]
test=test.fillna(0)
test['status']=test['status'].astype('int64')
test['status1']=test['status1'].astype('int64')
test['work']=test['work'].astype('int64')
test['work1']=test['work1'].astype('int64')


In [ ]:
adExposure = pd.read_csv('./adExposure.csv')
adExposure['msrequestTime']=pd.to_datetime(adExposure['requestTime'],unit='s')
adExposure['drequestTime'] = adExposure.msrequestTime.dt.date
import datetime
train = adExposure[adExposure['drequestTime'] > datetime.date(year=2019,month=3,day=14)]
train = train[train['drequestTime'] != datetime.date(year=2019,month=3,day=19)]
validate = adExposure[adExposure['drequestTime'] == datetime.date(year=2019,month=3,day=19)]
train = train.reset_index(drop=True)

In [7]:
Input= ['tid','aid','price', 'aIndustryId', 'createTime', 'pType',  'size', 
'pricepersize', 'bigprice', 'age', 'connectionType', 'consuptionAbility', 
        'device', 'education', 'gender', 'status','status1', 'work','work1','equality_ecpm', 'pctr','totalEcpm', 'basicecpm']


In [14]:
first_model_index = ['tid', 'aid','price', 'createTime','pType', 'size', 'aIndustryId', 'age', 'connectionType', 'consuptionAbility', 
                     'equality_ecpm', 'pctr','totalEcpm','device', 'education', 'gender', 'status', 'work']

In [15]:
train = pd.merge(train, adstatic, on='aid', how='left')
train = pd.merge(train, user, on='uid', how='left')
train = train.drop(['size_y','msrequestTime','drequestTime','auid'],axis = 1)
train.rename(columns={'size_x':'size', 'adPrice':'price', 'aRequestId':'tid'}, inplace=True)
train = train[first_model_index]
statuscut = train['status'].str.split(" ", expand = True)
workcut = train['work'].str.split(" ", expand = True) 
train['status']=statuscut[0]
train['status1']=statuscut[1]
train['work']=workcut[0]
train['work1']=workcut[1]
train=train.fillna(0)
train['status']=train['status'].astype('int64')
train['status1']=train['status1'].astype('int64')
train['work']=train['work'].astype('int64')
train['work1']=train['work1'].astype('int64')
train['pricepersize'] = train['price']/train['size'] * 100000000000000000
train['bigprice'] = train['price'] * 100000000000000000
train['basicecpm']=train['pctr']*train['price']
traininput = train.groupby(['aid','price'], as_index=False).mean()
trainoutput = train.groupby(['aid','price'], as_index=False).count()['tid']


In [ ]:
test1 = pd.merge(test, traininput[['aid','price', 'equality_ecpm', 'pctr','totalEcpm']], on=['aid','price'], how='left').drop_duplicates(subset=['aid','price'])
testinput = test1.fillna(0)
testinput = testinput.reset_index(drop = True)
testinput['basicecpm']=testinput['pctr']*testinput['price']

In [18]:
validate = validate.reset_index(drop=True)
validate = pd.merge(validate, adstatic, on='aid', how='left')
validate = pd.merge(validate, user, on='uid', how='left')
validate.rename(columns={'size_x':'size', 'adPrice':'price', 'aRequestId':'tid'}, inplace=True)
validate = validate.drop(['size_y','msrequestTime','drequestTime','auid'],axis = 1)
validate = validate[first_model_index]
statuscut = validate['status'].str.split(" ", expand = True)
workcut = validate['work'].str.split(" ", expand = True) 
validate['status']=statuscut[0]
validate['status1']=statuscut[1]
validate['work']=workcut[0]
validate['work1']=workcut[1]
validate=validate.fillna(0)
validate['status']=validate['status'].astype('int64')
validate['status1']=validate['status1'].astype('int64')
validate['work']=validate['work'].astype('int64')
validate['work1']=validate['work1'].astype('int64')
validate['pricepersize'] = validate['price']/train['size'] * 100000000000000000
validate['bigprice'] = validate['price'] * 100000000000000000
validate['basicecpm']=validate['pctr']*validate['price']
traininput = train.groupby(['aid','price'], as_index=False).mean()
validateinput = validate.groupby(['aid','price'], as_index=False).mean()
validateoutput = validate.groupby(['aid','price'], as_index=False).count()['tid']

In [8]:
reg = LinearRegression()
reg.fit(traininput[Input], trainoutput)
y_pred1 = reg.predict(validateinput[Input]) 
pred1 = reg.predict(testinput[Input]) 

In [ ]:
regtree2 = decision_tree_regression.create(traininput[Input], trainoutput)
y_pred22 = regtree2.predict(validateinput[Input])
results = model.evaluate(validateinput[Input])
results

In [ ]:
model.show(view="Tree", tree_id=0)
# Regression Tree
regtree = DecisionTreeRegressor(random_state = 0)
regtree.fit(traininput[Input], trainoutput)
y_pred2 = regtree.predict(validateinput[Input]) 
# pred2 = regtree.predict(testinput[Input]) 

In [9]:
# from sklearn.tree import export_graphviz
# from IPython.display import SVG
# from IPython.display import display
# from graphviz import Source
# treevisiual = export_graphviz(regtree, out_file = None,
#                                    feature_names = Input, filled= True,
#                                    rounded = True, special_characters = True)
# display(SVG(graph.pipe(format='svg')))

In [ ]:
# Random forsest
RFreg = RandomForestRegressor(n_estimators = 10, criterion = 'mse', random_state = 0)
RFreg.fit(traininput[Input], trainoutput)
y_pred3 = RFreg.predict(validateinput) 
pred3 = RFreg.predict(testinput[Input]) 
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(regtree, out_file = dot_data, 
                filled = True, rounded = True, special_characters = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

In [10]:
data_dmatrix = xgb.DMatrix(data=traininput[Input],label=trainoutput)
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.2, learning_rate = 0.01,
                max_depth = 10, alpha = 10, n_estimators = 10)
xg_reg.fit(traininput[Input],trainoutput)

y_pred4 = xg_reg.predict(validateinput[Input])
pred4 = xg_reg.predict(testinput[Input]) 
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

In [ ]:
# Light gbm
lgb_train = lgb.Dataset(traininput[Input], trainoutput)
lgb_eval = lgb.Dataset(validateinput[Input], validateoutput, reference = lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)
y_pred5 = gbm.predict(validateinput, num_iteration=gbm.best_iteration) 
pred5 = gbm.predict(testinput[Input]) 

In [ ]:
lgb.plot_importance(gbm)
testinput

In [12]:
y_pred1 = y_pred1 + alpha * validateinput['price']
y_pred2 = y_pred2 + alpha * validateinput['price']
y_pred3 = y_pred3 + alpha * validateinput['price']
y_pred4 = y_pred4 + alpha * validateinput['price']

In [219]:
# SMPAE
gap1 = np.absolute(y_pred1-validateoutput.values)
Sum1 = y_pred1+validateoutput.values
gap2 = np.absolute(y_pred2-validateoutput.values)
Sum2 = y_pred2+validateoutput.values
gap3 = np.absolute(y_pred3-validateoutput.values)
Sum3 = y_pred3+validateoutput.values
gap4 = np.absolute(y_pred4-validateoutput.values)
Sum4 = y_pred4+validateoutput.values
gap5 = np.absolute(y_pred5-validateoutput.values)
Sum5 = y_pred5+validateoutput.values

smpae1=2*gap1/Sum1
smpae2=2*gap2/Sum2
smpae3=2*gap3/Sum3
smpae4=2*gap4/Sum4
smpae5=2*gap5/Sum5


SMPAE1= smpae1.mean()
SMPAE2= smpae2.mean()
SMPAE3= smpae3.mean()
SMPAE4= smpae4.mean()
SMPAE5= smpae5.mean()

print('reg SMPAE : {}'.format(SMPAE1),'regtree SMPAE : {}'.format(SMPAE2))
print('RF SMPAE : {}'.format(SMPAE3),'xgb SMPAE : {}'.format(SMPAE4))
print('lgb SMPAE : {}'.format(SMPAE5))

reg SMPAE : 1.011671552009012 regtree SMPAE : 0.22874564768780326
RF SMPAE : 0.8395390879796779 xgb SMPAE : 0.5071402324466373
lgb SMPAE : 0.8155647199545979


In [220]:
validateinput['pred1']=y_pred1 #reg
validateinput['pred2']=y_pred2 #regtree
validateinput['pred3']=y_pred3 #rf
validateinput['pred4']=y_pred4 #xgb
validateinput['pred5']=y_pred5 #lgb
out = validateinput[['aid','price','pred1','pred2','pred3','pred4','pred5']]
aid = out['aid'].unique()

S1 = []
S2 = []
S3 = []
S4 = []
S5 = []
for a in aid :
    s1 = []
    s2 = []
    s3 = []
    s4 = []
    s5 = []
    each = out[out['aid']==a].reset_index(drop=True)
    price0 = each['price'][0]
    pred10 = each['pred1'][0]
    pred20 = each['pred2'][0]
    pred30 = each['pred3'][0]
    pred40 = each['pred4'][0]
    pred50 = each['pred5'][0]
    
    num = each.count()[0]
    for n in range(num) :
        b1 = (each['price'][n]-price0)*(each['pred1'][n]-pred10)
        b2 = (each['price'][n]-price0)*(each['pred2'][n]-pred20)
        b3 = (each['price'][n]-price0)*(each['pred3'][n]-pred30)
        b4 = (each['price'][n]-price0)*(each['pred4'][n]-pred40)
        b5 = (each['price'][n]-price0)*(each['pred5'][n]-pred40)
        if b1 > 0.0 :
            s1.append(1)
        elif b1 < 0.0 :
            s1.append(-1)
        else :
            s1.append(0)
        if b2 > 0.0 :
            s2.append(1)
        elif b2 < 0.0 :
            s2.append(-1)
        else :
            s2.append(0)
        if b3 > 0.0 :
            s3.append(1)
        elif b3 < 0.0 :
            s3.append(-1)
        else :
            s3.append(0)
        if b4 > 0.0 :
            s4.append(1)
        elif b4 < 0.0 :
            s4.append(-1)
        else :
            s4.append(0)
        if b5 > 0.0 :
            s5.append(1)
        elif b5 < 0.0 :
            s5.append(-1)
        else :
            s5.append(0)
            
    S1.append(sum(s1)/len(s1))
    S2.append(sum(s2)/len(s2))
    S3.append(sum(s3)/len(s3))
    S4.append(sum(s4)/len(s4))
    S5.append(sum(s5)/len(s5))    

monoscore1 = sum(S1)/len(S1)
monoscore2 = sum(S2)/len(S2)
monoscore3 = sum(S3)/len(S3)
monoscore4 = sum(S4)/len(S4)
monoscore5 = sum(S5)/len(S5)

print('reg Monoscore : {}'.format(monoscore1),'regtree Monoscore : {}'.format(monoscore2))
print('RF Monoscore : {}'.format(monoscore3),'xgb Monoscore : {}'.format(monoscore4))
print('lgb Monoscore : {}'.format(monoscore5))

reg Monoscore : 0.2903712356259061 regtree Monoscore : 0.39490247792691596
RF Monoscore : 0.08945793788348683 xgb Monoscore : 0.302729492243552
lgb Monoscore : 0.6145628505424909


In [221]:
w1=40
w2=60
TotalScore1 = w1*(1-SMPAE1/2)+w2*(monoscore1 + 1)/2
TotalScore2 = w1*(1-SMPAE2/2)+w2*(monoscore2 + 1)/2
TotalScore3 = w1*(1-SMPAE3/2)+w2*(monoscore3 + 1)/2
TotalScore4 = w1*(1-SMPAE4/2)+w2*(monoscore4 + 1)/2
TotalScore5 = w1*(1-SMPAE5/2)+w2*(monoscore5 + 1)/2

print('reg TotalScore : {}'.format(TotalScore1),'regtree TotalScore : {}'.format(TotalScore2))
print('RF TotalScore : {}'.format(TotalScore3),'xgb TotalScore : {}'.format(TotalScore4))
print('lgb TotalScore : {}'.format(TotalScore5))

reg TotalScore : 58.47770602859694 regtree TotalScore : 77.2721613840514
RF TotalScore : 55.892956376911044 xgb TotalScore : 68.93908011837382
lgb TotalScore : 72.12559111718278


In [ ]:
testinput
testinput['pred'] = pred2
pred = testinput[['tid', 'pred']]
pred['pred'] = pred['pred'].round(4)
pred.pred = pred.pred.astype(float)
pred.to_csv('submission.csv', index=False)